In [1]:
!nltk.download('stopwords')

import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer
import pymorphy2
from pymystem3 import Mystem
from string import punctuation

stop_words = stopwords.words('russian')
stop_words.remove("не")
stop_words.remove("ни")
mystem = Mystem()
stemmer = SnowballStemmer(language='russian')
morph = pymorphy2.MorphAnalyzer()

/bin/bash: -c: line 0: syntax error near unexpected token `'stopwords''
/bin/bash: -c: line 0: `nltk.download('stopwords')'


In [2]:
text = "Екатери́на I (Ма́рта Самуи́ловна Скавро́нская, в браке Крузе; после принятия православия Екатери́на Алексе́евна Миха́йлова; 5 [15] апреля 1684 — 6 [17] мая 1727, Санкт-Петербург) — российская императрица с 1721 года (как супруга царствующего императора)"

def preprocess(text, is_stem = False, is_morph = True, is_remove_stop_words = True):
    if text is None:
        text = ""
    text = text.lower()
    text = re.sub(r'\[.*?\]|\(.*?\)|\n|,|\.|\xa0|;', "", text).replace(chr(769), "")
    # tokens = word_tokenize(text, language='russian')
    tokens = mystem.lemmatize(text)
    if is_remove_stop_words:
        tokens = [token for token in tokens if token not in stop_words\
            and token != " " and token != '' \
            and token.strip() not in punctuation]

    if is_stem:
        tokens = list(map(lambda x: stemmer.stem(x), tokens))
    if is_morph:
        tokens = list(map(lambda x: morph.parse(x)[0].normal_form, tokens))

    return ' '.join(tokens)

preprocess(text)


# PREPROCESS FOR LABELING

'екатерина i   —  российский императрица 1721 год'

In [15]:
import pandas as pd
origin_result_data = pd.read_csv("../spacy/data/result.csv")
origin_result_data.head()

,Unnamed: 0,original,scored_text,distance,score
0,0,"Вели́кое кня́жество Влади́мирское, до 1157года...",С середины XIII века сюзеренитет великих княз...,0.923924,5
1,1,Новгоро́дская респу́блика — севернорусское сре...,Находясь с 1245 года под сюзеренитетом велики...,0.908120,5
2,2,Вели́кое кня́жество Лито́вское— восточноевропе...,С 1385 года находилось в личной унии с Королев...,0.861883,5
3,3,Вели́кое кня́жество Моско́вское— средневековое...,Возвышению Москвы и укреплению её авторитета ...,0.901416,5
4,4,Оле́г — князь Новгородский с 879 года и велики...,Получив власть над новгородскими землями посл...,0.916509,5


In [16]:
def preprocess_for_labeling(data):
    after_preprocess = data["original"].transform(preprocess)

    after_preprocess = after_preprocess.dropna()
    after_preprocess = after_preprocess.drop_duplicates()

    with open('/home/droman/Documents/diploma/spacy/data/data_for_labeling_with_preprocess.txt', mode='wt', encoding='utf-8') as myfile:
        myfile.write('\n'.join(set(after_preprocess.tolist())))


def preprocess_for_classification(data):
    for title in ["original", "scored_text"]:
        after_preprocess = data[title].transform(preprocess)

        after_preprocess = after_preprocess.dropna()
        after_preprocess = after_preprocess.drop_duplicates()

        data[title] = after_preprocess

    data.to_csv('/home/droman/Documents/diploma/deeppavlov_ner_3.6/data/data_with_preprocess.csv')

In [17]:
preprocess_for_classification(origin_result_data)


In [18]:
pd.read_csv("./data/data_with_preprocess.csv").head()

,Unnamed: 0,Unnamed: 0.1,original,scored_text,distance,score
0,0,0,великий княжество владимирский 1157год — суз...,середина xiii век сюзеренитет великий князь вл...,0.923924,5
1,1,1,новгородский республика — севернорусский сре...,находиться 1245 год сюзеренитет великий князь ...,0.908120,5
2,2,2,великий княжество литовский — восточноевропей...,1385 год находиться личный уния королевство по...,0.861883,5
3,3,3,великий княжество московский — средневековый ...,возвышение москва укрепление авторитет русь сп...,0.901416,5
4,4,4,олег — князь новгородский 879 год великий кн...,получать власть новгородский земля смерть рюри...,0.916509,5
